# Object Detection Pipeline - Hyperparameter Tuning & Model Training

Hyperparameter tuning with Ray Tune on validation set, followed by final training on train+val and evaluation on test set.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kshitijrajsharma/dl4cv-oda/blob/master/notebooks/experiment.ipynb)

In [1]:
# ! pip install dl4cv_oda ray[tune]

In [2]:
import os
import yaml
import shutil
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from dl4cv_oda import (clean_osm_data, clip_labels_to_tiles, convert_to_yolo_format,
                       create_train_val_split, create_yolo_config, download_tiles)
from ultralytics import YOLO, RTDETR

## Configuration

In [ ]:
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
CHIPS_DIR = DATA_DIR / "chips"
LABELS_DIR = DATA_DIR / "labels"
YOLO_DIR = DATA_DIR / "yolo"

OSM_FILE = RAW_DIR / "kolovai-trees.geojson"
CLEANED_FILE = RAW_DIR / "cleaned.geojson"
TREES_BOX_FILE = DATA_DIR / "trees_box.geojson"
TILES_FILE = DATA_DIR / "tiles.geojson"

IMG_SIZE = 256
EPOCHS = 200
PATIENCE = 10

TUNE_EPOCHS = 30
TUNE_ITERATIONS = 3
GPU_PER_TRIAL = 1

MODELS_TO_TRAIN = {
    "yolov8l": "yolov8l.pt",
    "yolo12l": "yolo12l.pt",
    "rtdetr-l": "rtdetr-l.pt"
}

In [4]:
import requests
if not OSM_FILE.exists():
    OSM_FILE.parent.mkdir(parents=True, exist_ok=True)
    OSM_FILE.write_bytes(requests.get("https://github.com/kshitijrajsharma/dl4cv-oda/blob/master/data/raw/kolovai-trees.geojson?raw=true", allow_redirects=True).content)
    print(f"Downloaded {OSM_FILE}")
else:
    print("OSM data ready")

OSM data ready


## Step 1: Clean OSM Data

In [5]:
if not CLEANED_FILE.exists():
    count = clean_osm_data(str(OSM_FILE), str(CLEANED_FILE), str(TREES_BOX_FILE))
    print(f"Processed {count} trees")
else:
    print("Cleaned data ready")

Cleaned data ready


## Download Tiles

In [6]:
if not TILES_FILE.exists():
    data = gpd.read_file(TREES_BOX_FILE)
    data.to_crs(epsg=4326, inplace=True)
    bbox = list(data.total_bounds)
    await download_tiles(bbox, 19, "https://tiles.openaerialmap.org/5a28639331eff4000c380690/0/5b1b6fb2-5024-4681-a175-9b667174f48c/{z}/{x}/{y}.png", DATA_DIR, 'OAM')
    print("Tiles downloaded")

## Clip Labels & Convert to YOLO Format

In [7]:
if TILES_FILE.exists() and not (YOLO_DIR / "train").exists():
    stats = clip_labels_to_tiles(str(TREES_BOX_FILE), str(TILES_FILE), str(LABELS_DIR))
    print(f"{stats['processed']} tiles, {stats['total_trees']} trees")
    class_mapping = convert_to_yolo_format(str(TREES_BOX_FILE), str(CHIPS_DIR), str(LABELS_DIR), str(YOLO_DIR), target_species="Coconut")
    print(f"Class mapping: {class_mapping}")
else:
    print("YOLO format data ready")

YOLO format data ready


## Create Train/Val/Test Split

In [8]:
if not (YOLO_DIR / "train").exists():
    train_count, val_count, test_count = create_train_val_split(str(LABELS_DIR), str(CHIPS_DIR), str(YOLO_DIR), train_ratio=0.7, val_ratio=0.2, test_ratio=0.1)
    print(f"Train: {train_count} | Val: {val_count} | Test: {test_count}")
else:
    print("Train/Val/Test split ready")

config_file = create_yolo_config(str(YOLO_DIR), {"Coconut": 0})
config_file_abs = os.path.abspath(config_file)
print(f"Config: {config_file_abs}")

Train/Val/Test split ready
Config: /home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/data/yolo/config.yaml


## Hyperparameter Tuning with Ray Tune

Tune all models with consistent parameters.

In [ ]:
tune_results = {}
for model_name, model_weights in MODELS_TO_TRAIN.items():
    print(f"Tuning {model_name}...")
    model = RTDETR(model_weights) if "rtdetr" in model_name.lower() else YOLO(model_weights)
    result_grid = model.tune(data=config_file_abs, epochs=TUNE_EPOCHS, imgsz=IMG_SIZE, 
                            patience=PATIENCE, iterations=TUNE_ITERATIONS, 
                            gpu_per_trial=GPU_PER_TRIAL, use_ray=True)
    tune_results[model_name] = result_grid
    print(f"{model_name} tuning complete")

(_tune pid=3475307) New https://pypi.org/project/ultralytics/8.3.252 available 😃 Update with 'pip install -U ultralytics'
(_tune pid=3475307) Ultralytics 8.3.250 🚀 Python-3.13.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4090 Laptop GPU, 15944MiB)
(_tune pid=3475307) engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.907854238249426, box=0.07963073825918983, cache=False, cfg=None, classes=None, close_mosaic=10, cls=3.5855302095415205, compile=False, conf=None, copy_paste=0.09889050143185318, copy_paste_mode=flip, cos_lr=False, cutmix=0.05693824651467838, data=/home/krschap/academia/dl4cv-object-detection-on-aerial-imagery/data/yolo/config.yaml, degrees=13.775788821871268, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.6703543885383698, flipud=0.04201215309291839, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0522778427259

## Analyze Results & Extract Best Configs

In [ ]:
def analyze_and_save_results(result_grid, model_name):
    results_list = []
    for i, result in enumerate(result_grid):
        df = result.metrics_dataframe
        p = df['metrics/precision(B)'].iloc[-1] if 'metrics/precision(B)' in df else 0
        r = df['metrics/recall(B)'].iloc[-1] if 'metrics/recall(B)' in df else 0
        m = df['metrics/mAP50(B)'].iloc[-1] if 'metrics/mAP50(B)' in df else 0
        f1 = 2 * (p * r) / (p + r + 1e-6) if (p + r) > 0 else 0
        results_list.append({'trial': i, 'config': result.config, 'precision': p, 'recall': r, 'mAP50': m, 'f1': f1})
    
    results_df = pd.DataFrame(results_list).sort_values('f1', ascending=False)
    best = results_df.iloc[0]
    best_config = best['config']
    
    with open(YOLO_DIR / f"best_config_{model_name}.yaml", 'w') as f:
        yaml.dump(best_config, f, default_flow_style=False)
    
    print(f"{model_name:12s} F1: {best['f1']:.4f} | P: {best['precision']:.4f} | R: {best['recall']:.4f} | mAP50: {best['mAP50']:.4f}")
    return results_df, best_config

best_configs = {}
for model_name, result_grid in tune_results.items():
    _, best_configs[model_name] = analyze_and_save_results(result_grid, model_name)

## Merge Train+Val for Final Training

In [ ]:
trainval_dir = YOLO_DIR / "trainval"
trainval_config = YOLO_DIR / "data_trainval.yaml"

if not trainval_dir.exists():
    trainval_dir.mkdir(exist_ok=True)
    for split in ["train", "val"]:
        for f in (YOLO_DIR / split).glob("*"):
            shutil.copy(f, trainval_dir / f.name)
    with open(config_file, 'r') as f:
        trainval_config.write_text(f.read().replace("train: train", "train: trainval"))
    print(f"Merged train+val into {trainval_dir}")
else:
    print("Train+val merge ready")

## Final Training with Best Hyperparameters

In [ ]:
trained_models = {}
for model_name, model_weights in MODELS_TO_TRAIN.items():
    print(f"\nTraining {model_name} on train+val...")
    model = RTDETR(model_weights) if "rtdetr" in model_weights.lower() else YOLO(model_weights)
    cfg = best_configs[model_name].copy()
    cfg.update({'data': str(trainval_config), 'epochs': EPOCHS, 'imgsz': IMG_SIZE, 
                'patience': PATIENCE, 'name': f"final_{model_name}", 'plots': True})
    model.train(**cfg)
    trained_models[model_name] = model
    print(f"{model_name} complete")

## Evaluate on Test Set

In [ ]:
test_results = {}
for name, model in trained_models.items():
    print(f"Evaluating {name}...")
    m = model.val(split='test')
    p, r = m.box.mp, m.box.mr
    test_results[name] = {"precision": p, "recall": r, "f1": 2 * (p * r) / (p + r + 1e-6),
                         "mAP50": m.box.map50, "mAP50-95": m.box.map}

In [ ]:
results_df = pd.DataFrame(test_results).T.sort_values('f1', ascending=False)
print("Test Set Results:")
print(results_df.to_string())
print(f"\nBest: {results_df.index[0]}")

## F1 Score per Epoch

In [ ]:
n = len(MODELS_TO_TRAIN)
cols = min(3, n)
rows = (n + cols - 1) // cols
fig, axes = plt.subplots(rows, cols, figsize=(6*cols, 5*rows))
axes = axes.flatten() if n > 1 else [axes]

for idx, model_name in enumerate(MODELS_TO_TRAIN.keys()):
    csv = Path(f"runs/detect/final_{model_name}/results.csv")
    if csv.exists():
        df = pd.read_csv(csv)
        df.columns = df.columns.str.strip()
        p, r = df['metrics/precision(B)'], df['metrics/recall(B)']
        f1 = 2 * (p * r) / (p + r + 1e-6)
        axes[idx].plot(f1, label='F1', linewidth=2, color='#2E86AB')
        axes[idx].plot(p, label='Precision', alpha=0.7, color='#A23B72')
        axes[idx].plot(r, label='Recall', alpha=0.7, color='#F18F01')
        axes[idx].set_title(model_name.upper(), fontsize=11, fontweight='bold')
        axes[idx].set_xlabel('Epoch')
        axes[idx].set_ylabel('Score')
        axes[idx].legend()
        axes[idx].grid(True, alpha=0.3)

for idx in range(n, len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.savefig('f1_scores_comparison.png', dpi=150, bbox_inches='tight')
plt.show()